# **Data Modelling and Evaluating**
---

## Objectives

* Answer business requirement 2: 
    * The client wants the system to identify which animal species is present in an image.


## Inputs

* inputs/datasets/animals/image/train/
* inputs/datasets/animals/image/test/
* inputs/datasets/animals/image/validation/
* Image shape embeddings: outputs/v1/image_shape.pkl

## Outputs

* Images distribution plot in train, validation, and test set.
* Image augmentation pipeline.
* Class indices mapping.
* Machine learning model creation and training.
* Saved trained model.
* Learning curve plots for model performance.
* Model evaluation stored in pickle file.
* Prediction on a random image file.


## Additional Comments | Insights | Conclusions

* N/A for now

---

## Import Packages

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import joblib
sns.set_style("white")

## Set Working Directory

In [2]:
cwd = os.getcwd()
os.chdir('/workspaces/Animal_detection_camera')
print("You set a new current directory")

work_dir = os.getcwd()
work_dir

You set a new current directory


'/workspaces/Animal_detection_camera'

## Set input directories
Set train, validation and test paths

In [7]:
my_data_dir = 'inputs/datasets/animals/image'
train_path = my_data_dir + '/train'
val_path = my_data_dir + '/validation'
test_path = my_data_dir + '/test'


## Set output directory

In [8]:
version = 'v1'
file_path = f'outputs/{version}'
os.makedirs(file_path, exist_ok=True)

if 'outputs' in os.listdir(work_dir) and version in os.listdir(work_dir + '/outputs'):
    print('Old version is already available, create a new version.')
else:
    os.makedirs(name=file_path, exist_ok=True)

Old version is already available, create a new version.


## Set labels

In [10]:
labels = os.listdir(train_path)

print(
    f"Project Labels: {labels}"
)

Project Labels: ['lemur', 'snake', 'elephant', 'frog', 'chimpanzee', 'chinchilla', 'flamingo', 'mongoose', 'ostrich', 'ferret', 'camel', 'bee', 'mole', 'penguin', 'leopard', 'hawk', 'hedgehog', 'walrus', 'falcon', 'grasshopper', 'beaver', 'antelope', 'giraffe', 'duck', 'lizard', 'crab', 'goose', 'gorilla', 'jaguar', 'sheep', 'lynx', 'butterfly', 'panda', 'goat', 'deer', 'peacock', 'dog', 'whale', 'kangaroo', 'seal', 'cheetah', 'cow', 'iguana', 'hippopotamus', 'fox', 'cat', 'donkey', 'raccoon', 'blackbird', 'buffalo', 'koala', 'crocodile', 'dolphin', 'hyena', 'porcupine', 'bear', 'squid', 'spider', 'eagle', 'bison', 'owl', 'otter', 'snail', 'wolf']
